In [41]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from joblib import dump
import numpy as np

In [23]:
vgg19_model = VGG19(weights='imagenet', include_top=False, input_shape=(128,128,3))

In [24]:
vgg19_model.trainable = False

In [25]:
model = Sequential()
model.add(vgg19_model)
model.add(Flatten())
model.add(Dense(units=256,activation='relu'))
model.add(Dense(units=3,activation='softmax'))

In [26]:
training_data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [27]:
testing_data_generator = ImageDataGenerator(
    rescale= 1./255
)

In [28]:
train_path = 'Covid-19/train'
test_path = 'Covid-19/test'

In [37]:
training_dataset = training_data_generator.flow_from_directory(
    train_path,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical'
)

Found 251 images belonging to 3 classes.


In [38]:
testing_dataset = testing_data_generator.flow_from_directory(
    test_path,
    target_size=(128,128),
    batch_size=32,
    class_mode='categorical'
)

Found 66 images belonging to 3 classes.


In [39]:
model.compile(optimizer='adam' , loss='categorical_crossentropy' , metrics='accuracy')

In [40]:
model.fit(training_dataset , epochs=15 , verbose = 1)

Epoch 1/15
8/8 [==============================] - 9s 1s/step - loss: 0.2826 - accuracy: 0.8765
Epoch 2/15
8/8 [==============================] - 11s 1s/step - loss: 0.1964 - accuracy: 0.9084
Epoch 3/15
8/8 [==============================] - 25s 3s/step - loss: 0.1415 - accuracy: 0.9522
Epoch 4/15
8/8 [==============================] - 25s 3s/step - loss: 0.0953 - accuracy: 0.9721
Epoch 5/15
8/8 [==============================] - 26s 3s/step - loss: 0.0861 - accuracy: 0.9641
Epoch 6/15
8/8 [==============================] - 26s 3s/step - loss: 0.0769 - accuracy: 0.9801
Epoch 7/15
8/8 [==============================] - 28s 3s/step - loss: 0.0710 - accuracy: 0.9801
Epoch 8/15
8/8 [==============================] - 26s 3s/step - loss: 0.0745 - accuracy: 0.9681
Epoch 9/15
8/8 [==============================] - 25s 3s/step - loss: 0.0530 - accuracy: 0.9920
Epoch 10/15
8/8 [==============================] - 24s 3s/step - loss: 0.0576 - accuracy: 0.9880
Epoch 11/15
8/8 [=======================

In [42]:
def preprocess_testing_image(image_path, target_size=(128, 128)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0 
    return img_array

In [44]:
image_path = 'Covid-19/test/Covid/COVID-00003b.jpg'
preprocessed_img = preprocess_testing_image(image_path)

In [45]:
predictions = model.predict(preprocessed_img)

# Decode the prediction
predicted_class = np.argmax(predictions)
class_names = training_dataset.class_indices
predicted_class_name = [k for k, v in class_names.items() if v == predicted_class][0]

print("Predicted class:", predicted_class_name)

1/1 [==============================] - 0s 323ms/step
Predicted class: Covid


In [46]:
dump(model,'trained model.pkl')

['trained model.pkl']